# Código

In [3]:
import pandas as pd

In [6]:
df = pd.read_csv('data/dataset_pishing.csv')

In [8]:
# 5 entries
df.head(5)

,url,status
0,http://www.crestonwood.com/router.php,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,phishing
2,https://support-appleld.com.secureupdate.duila...,phishing
3,http://rgipt.ac.in,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,legitimate


In [11]:
# count of legit and phishing
df['status'].value_counts()

status
legitimate    5715
phishing      5715
Name: count, dtype: int64

El dataset está balanceado. 

1. **¿Qué ventajas tiene el análisis de una URL contra el análisis de otros datos, cómo el tiempo de vida del dominio, o las características de la página Web?**
Analizar las url tiene muchas ventajas, entre ellas:
- Evita la descarga del sitio, el cual podría contener código malicioso.
- Es mucho más rapida al detectar en tiempo real.
- Es computacionalmente más eficaz ya que solo se deben procesar strings cortos.
2. **¿Qué características de una URL son más prometedoras para la detección de phishing?**
- Longitud de la url completa
- Longitud del dominio
- Cantidad de subdominios
- Cantidad o presencia de caracteres especiales
- Cantidad de números
- Cantidad de letras
- Protocolo
- Cantidad de directorios
- Presencia de parámetros
- Cantidad de párametros
- IP en la url
- TDL
- Identificación de redireccónes (posee más de una url)
- Presencia de credenciales en la url (correo)
- Entropía

In [ ]:
# get the url length
df['url_length'] = df['url'].apply(lambda x: len(x))

In [ ]:
# get the domain length
df['domain_length'] = df['url'].apply(lambda x: len(x.split('/')[2]))

In [ ]:
# get protocol
df['protocol'] = df['url'].apply(lambda x: x.split(':')[0])

In [ ]:
# get the number of subdomains
df['subdomains'] = df['url'].apply(lambda x: len(x.split('/')[2].split('.')) - 2)

In [ ]:
# get if the url has a special characters
special_characters = ['@', '-', '_', '%', '?', '=', '&', '.']
df['special_characters'] = df['url'].apply(lambda x: 1 if any(c in x for c in special_characters) else 0)

In [ ]:
# get the number of special characters
df['special_characters_count'] = df['url'].apply(lambda x: sum(1 for c in x if c in special_characters))

In [ ]:
# get the number of digits
df['digits'] = df['url'].apply(lambda x: sum(c.isdigit() for c in x))

In [ ]:
# get the number of letters
df['letters'] = df['url'].apply(lambda x: sum(c.isalpha() for c in x))

In [ ]:
# get the number of directories, 
def get_directories(url: str):
    domain_and_directories = url.split('://', 1)[1].split('?')[0]
    directories = domain_and_directories.split('/')[1:]
    cleaned_directories = [d for d in directories if d != '']
    return cleaned_directories

In [ ]:
def has_file_extension(url):
    dirs = get_directories(url)
    return any('.' in d for d in dirs)

In [12]:
# get if the url has a file extension
df['has_file_extension'] = df['url'].apply(lambda x: 1 if has_file_extension(df['url']) else 0)

NameError: name 'has_file_extension' is not defined

In [ ]:
def count_file_extensions(url):
    dirs = get_directories(url)
    return sum(1 for dir in dirs if '.' in dir)

In [ ]:

# get the number of file extensions
df['file_extension_count'] = df['url'].apply(count_file_extensions)